In [5]:
"Advanced circuits"

import numpy as np
from qiskit import *

from qiskit.circuit import Gate

"OPAQUE GATES"
#CRÉATION D'UNE NOUVELLE PORTE
my_gate = Gate(name='my_gate', num_qubits=2, params=[])

#un registre quantique permet de créer 3 qbit d'input, à quoi sert le 'q' ? 
qr = QuantumRegister(3, 'q')
#création d'un circuit quantique
circ = QuantumCircuit(qr)
#ajout de deux portes
circ.append(my_gate, [qr[0], qr[1]])
circ.append(my_gate, [qr[1], qr[2]])

circ.draw()

┌──────────┐            
q_0: ┤0         ├────────────
     │  my_gate │┌──────────┐
q_1: ┤1         ├┤0         ├
     └──────────┘│  my_gate │
q_2: ────────────┤1         ├
                 └──────────┘

In [18]:
"Composite Gates"
#qu'est ce que ça pourrait être ? 

# Build a sub-circuit
#on créé un registre avec 2 entrées
sub_q = QuantumRegister(2)
#on créé le circuit quantique de base
sub_circ = QuantumCircuit(sub_q, name='sub_circ')
#on ajoute une porte h sur l'entrée 0
sub_circ.h(sub_q[0])


#on ajoute une porte crz sur |0> puis |1>, qu'est ce que c'est ?
""" theta: The angle of the rotation.
    control_qubit: The qubit(s) used as the control.
    target_qubit: The qubit(s) targeted by the gate.
    label: The string label of the gate in the circuit.
    ctrl_state: The control state in decimal, or as a bitstring (e.g. '1').  Defaults to controlling on the '1' state."""

#L'angle de rotation est 1 , le bit de contrôle est sub_q[0] , le bit cible est sub_q[1]
sub_circ.crz(1, sub_q[0], sub_q[1])

#on ajoute une barrière #elle peut être vue dans la version décomposée 
#sub_circ.barrier()
#on ajoute un id ?

#on ajoute une porte identité
sub_circ.id(sub_q[1])

#on ajoute une porte u 
sub_circ.u(1, 2, -2, sub_q[0])



# Convert to a gate and stick it into an arbitrary place in the bigger circuit
sub_inst = sub_circ.to_instruction()

qr = QuantumRegister(3, 'q')
circ = QuantumCircuit(qr)
circ.h(qr[0])
circ.cx(qr[0], qr[1])
circ.cx(qr[1], qr[2])
circ.append(sub_inst, [qr[1], qr[2]])

circ.draw()

┌───┐                       
q_0: ┤ H ├──■────────────────────
     └───┘┌─┴─┐     ┌───────────┐
q_1: ─────┤ X ├──■──┤0          ├
          └───┘┌─┴─┐│  sub_circ │
q_2: ──────────┤ X ├┤1          ├
               └───┘└───────────┘

In [19]:
decomposed_circ = circ.decompose() # Does not modify original circuit
decomposed_circ.draw()

┌─────────┐                                     
q_0: ┤ U2(0,π) ├──■──────────────────────────────────
     └─────────┘┌─┴─┐     ┌───┐         ┌───────────┐
q_1: ───────────┤ X ├──■──┤ H ├────■────┤ U(1,2,-2) ├
                └───┘┌─┴─┐└───┘┌───┴───┐└───┬───┬───┘
q_2: ────────────────┤ X ├─────┤ Rz(1) ├────┤ I ├────
                     └───┘     └───────┘    └───┘

In [ ]:
"""Parametrized circuits"""

from qiskit.circuit import Parameter

#on ajoute un paramêtre qui est theta
theta = Parameter('θ')

n = 5

#circuit quantique 
qc = QuantumCircuit(5, 1)

#application d'une porte hadamard
qc.h(0)
for i in range(n-1):
    qc.cx(i, i+1)

#une barriere
qc.barrier()

#rotation selon l'axe z
qc.rz(theta, range(5))

#barrière
qc.barrier()

for i in reversed(range(n-1)):
    qc.cx(i, i+1)
qc.h(0)
qc.measure(0, 0)

qc.draw('mpl')
